<a href="https://colab.research.google.com/github/RC11-SkillsClass2022-23/22101933/blob/main/image_for_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Defining functions

In [ ]:
!pip install --upgrade moviepy

In [ ]:
import tensorflow as tf
import PIL
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Dropout, Dense, Softmax)
from tensorflow.keras.applications import mobilenet as _mobilenet
import random
import os
import numpy as np

In [ ]:
! git clone https://github.com/kenshohara/video-classification-3d-cnn-pytorch.git

In [ ]:
%cd video-classification-3d-cnn-pytorch/

In [ ]:
import json
import os
from moviepy.editor import *
from moviepy import *
import shutil

In [ ]:
from PIL import Image

In [ ]:
def load_image(img_file, target_size=(224,224)):
    X = np.zeros((1, *target_size, 3))
    X[0, ] = np.asarray(tf.keras.preprocessing.image.load_img(
        img_file, 
        target_size=target_size)
    )
    X = tf.keras.applications.mobilenet.preprocess_input(X)
    return X

def ensure_folder_exists(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
model = tf.keras.applications.mobilenet.MobileNet(
# The 3 is the three dimensions of the input: r,g,b.
  input_shape=(224, 224, 3), 
  include_top=False, 
  pooling='avg'
)

In [ ]:
def processFilmByFrames(filmName, filmPath, interval, model):
    features = []
    
    film = VideoFileClip(filmPath)
    nrFrames = int(film.duration/interval)
    for i in range(nrFrames):
        s = i*interval
        frame = film.get_frame(s)
        frame = Image.fromarray(frame,'RGB')
        frame.save('/content/drive/MyDrive/youtube/gb_img/temp.jpg')
        
        im = load_image('/content/drive/MyDrive/youtube/gb_img/temp.jpg')
        f = model.predict(im)[0]
        features.append({'film':filmPath, 'second': s, 'features': f})
    return features

In [ ]:
def processVideosByFrames(folder, interval, model):
    features = []
    
    videos = os.listdir(folder)
    for v in videos:
        if v.split('.')[-1] == 'mp4':
            vpath = os.path.join(folder,v)
            film = VideoFileClip(vpath)
            nrFrames = int(film.duration/interval)
            for i in range(nrFrames):
                s = i*interval
                frame = film.get_frame(s)
                frame = Image.fromarray(frame,'RGB')
                frame.save('/content/drive/MyDrive/youtube/gb_img/temp.jpg')
        
        im = load_image('/content/drive/MyDrive/youtube/gb_img/temp.jpg')
        f = model.predict(im)[0]
        features.append({'film':vpath, 'second': s, 'features': f})
    return features

In [ ]:
def processImage(imagePath, model):
    im = load_image(imagePath)
    f = model.predict(im)[0]
    return f

In [ ]:
def findFramesByImage(filmFeatures, imagePath, model):
    imF = processImage(imagePath, model)
    
    bestFrameDist = 100000000
    bestFrameFilm = None
    bestFrameSecond = 0
    
    for f in filmFeatures:
        dist = np.linalg.norm(f['features']-imF)
        if dist < bestFrameDist:
            bestFrameDist = dist
            bestFrameFilm = f['film']
            bestFrameSecond = f['second']
    
    return [bestFrameFilm, bestFrameSecond]

In [ ]:
def getFilmAroundFrame(frameInfo, radius):
    film = VideoFileClip(frameInfo[0])
    s = max(frameInfo[-1] - radius, 0)
    e = min(frameInfo[-1]+radius, film.duration)
    return film.subclip(s,e)

In [ ]:
def readAllOutput(outputFolder):
    list = []
    output_files = os.listdir(outputFolder)
    for i in output_files:
      list.append(i)
    
    return list

## Remenber to change out put folder and image path

In [ ]:
outputFolder = '/content/drive/MyDrive/action_films'
list = readAllOutput(outputFolder)

In [ ]:
def img_for_video(img_path):
  for name in list:
    filmpath = '/content/drive/MyDrive/action_films/' + str(name)
    # write path with name

    print(filmpath)

    features = processFilmByFrames('frame.mp4',filmpath, 10, model)
  
    features[0]

    match = findFramesByImage(features, img_path, model)

    match

    frameClip = getFilmAroundFrame(match, 1)

    output_path = '/content/drive/MyDrive/action_films_output/' + str(name)

    frameClip.write_videofile(output_path, audio_codec='aac')

In [ ]:
img_for_video('/content/drive/MyDrive/img/Mosque_image.jpg')